In [1]:
import json
from datasets.vqa_dataset_vqav2 import prep_ans
import re

In [2]:
stat_ques_list = ['/home/ngoc/data/vqav2/new_vqa2_train_question.json', '/home/ngoc/data/vqav2/new_vqa2_val_question.json']
stat_ann_list = ['/home/ngoc/data/vqav2/new_vqa2_train_annotation.json', '/home/ngoc/data/vqav2/new_vqa2_val_annotation.json']
path_files = stat_ques_list
stat_ques_list = []
for path_file in path_files:
    with open(path_file, 'r') as file:
        single_questions = json.load(file)
    stat_ques_list += single_questions
question_list = {}
for ques in stat_ques_list:
    question_list[ques["question_id"]] = ques["question"]

examples = []
path_files = stat_ann_list

for path_file in path_files:
    with open(path_file, 'r') as file:
        single_anns = json.load(file)
    examples += single_anns



In [3]:
ans2tok, tok2ans = {}, {}

ans_freq_dict = {}

for ans in examples:
    ans_proc = prep_ans(ans['multiple_choice_answer'])
    if ans_proc not in ans_freq_dict:
        ans_freq_dict[ans_proc] = 1
    else:
        ans_freq_dict[ans_proc] += 1
ans_freq_filter = ans_freq_dict.copy()

most_frequent_words = dict(sorted(ans_freq_dict.items(), key=lambda item: item[1], reverse=True)[:1000])
ans_freq_filter = most_frequent_words

for ans in ans_freq_filter:
    tok2ans[tok2ans.__len__()] = ans
    ans2tok[ans] = ans2tok.__len__()

In [4]:
question_type_to_idx = {
            'yes/no': 0,
            'color': 1,
            'object': 2,
            'number': 3,
            'location': 4,
            'action': 5,
            'other': 6,
            'human': 7,
            'sport': 8
            }
idx_to_question_type = {
            0: 'yes/no',
            1: 'color',
            2: 'object',
            3: 'number',
            4: 'location',
            5: 'action',
            6: 'other',
            7: 'human',
            8: 'sport'
            }

In [5]:
def question_type_processing(question_type):
    question_type = question_type.strip()
    question_type = question_type.split(':')[-1]
    question_type = question_type.lower()
    question_type = re.sub(r'\([^)]*\)', '', question_type)
    question_type = question_type.replace("'", "")
    question_type = question_type.replace(".", "")
    question_type = question_type.replace("`", "")
    question_type = question_type.replace(";", ",")
    question_type = question_type.replace('"', "")
    question_type = question_type.replace("question type: ", "")
    question_type = question_type.replace("question: ", "")
    question_type = question_type.replace("’", "")
    question_type = question_type.replace("[", "")
    question_type = question_type.replace("]", "")
    question_type = question_type.replace("-", "")
    question_type = question_type.replace(" or ", ", ")
    question_type = question_type.replace("+", ", ")
    question_type = question_type.replace(" , ", ", ")
    question_type = question_type.replace("|", " ")
    question_type = re.sub('\s+', ' ', question_type).strip()
    # # 
    for word in ["clothing", "object", "material", "shape", "food", "transportation", "pattern", "letter", "drink"]:
        if word in question_type:
            return "object"
    for word in ["age", "animal", "body parts", "gender", "body part", "person", "emotion", "relationship"]:
        if word in question_type:
            return "human"
    for word in ["weather", "season", "time", "unknown", "any", "not", "event", "geometry", "music", "description", 
                 "flavor", "sound", "taste", "other", "comparison", "team", "duration", "sport this response is incorrect please try again", "fitting",
                 "orientation", "facial feature","please provide the question type for the question what does it say after water lane?"]:        
        if word in question_type:
            return "other"
    for word in ["activity", "action"]:        
        if word in question_type:
            return "action"
    for word in ["direction"]:        
        if word in question_type:
            return "location"
    for word in ["yes/no", "binary", "you missed providing an answer for this question",
                 "it seems like there was an error in the provided question please rephrase the question so that it can be categorized accurately",
                 "Is there traffic on the table?", "no"
                 ]:        
        if word in question_type:
            return "yes/no"
    for word in ["number", "you didnt provide a clear question so i am unable to categorize it into one of the question types provided"]:        
        if word in question_type:
            return "number"
    for word in ["sports"]:   
        if word in question_type:
            return "sport"
    for word in ["color"]:   
        if word in question_type:
            return "color"
    for word in ["location", "hand dominance"]:   
        if word in question_type:
            return "location"
    # question_type = question_type.split(",")
    # question_type =  [item.strip() for item in question_type][0]
    return question_type

In [6]:
question_type_dict = {}
with open("/home/ngoc/githubs/chatgpt-hie-vqa/datasets/train_question_type_gpt_v6_vqav2.json", 'r') as file:
    for line in file:
        question_object = json.loads(line)
        question_str = question_object["question"]
        question_type = question_object["question_type"]
        question_type = question_type_processing(question_type)
        question_type_dict[question_str] = question_type

In [7]:
from collections import defaultdict

# Initialize counters
answer_question_type_counts = defaultdict(lambda: defaultdict(int))
total_answer_counts = defaultdict(int)

for example in examples:
    unique_answers = {answer['answer'] for answer in example['answers']}
    for answer_str in unique_answers:
        ans_proc = prep_ans(answer_str)
        if ans_proc in ans2tok:
            ans_id = ans2tok[ans_proc]
            total_answer_counts[ans_id] += 1  # Total appearances of each answer
            
            question_str = question_list[example["question_id"]]
            question_type_str = question_type_dict[question_str]
            question_type_id = question_type_to_idx[question_type_str]
            
            # Count appearances of each answer across different question types
            answer_question_type_counts[ans_id][question_type_id] += 1

In [8]:
answer_percentages = defaultdict(dict)

for ans_id, question_type_counts in answer_question_type_counts.items():
    total_appearances = total_answer_counts[ans_id]
    for question_type_id, count in question_type_counts.items():
        percentage = (count / total_appearances)
        answer_percentages[ans_id][question_type_id] = round(percentage, 2)

In [9]:
answer_percentages

defaultdict(dict,
            {18: {2: 0.51,
              0: 0.02,
              8: 0.31,
              5: 0.09,
              4: 0.02,
              6: 0.02,
              7: 0.01,
              1: 0.01,
              3: 0.0},
             0: {0: 0.9,
              2: 0.02,
              4: 0.04,
              6: 0.01,
              1: 0.01,
              7: 0.01,
              3: 0.0,
              5: 0.0,
              8: 0.0},
             178: {1: 0.01,
              2: 0.85,
              6: 0.04,
              4: 0.08,
              0: 0.01,
              7: 0.01,
              5: 0.0,
              3: 0.0,
              8: 0.0},
             50: {1: 0.06,
              2: 0.67,
              6: 0.1,
              0: 0.02,
              4: 0.12,
              8: 0.03,
              7: 0.0,
              5: 0.0},
             1: {0: 0.92,
              4: 0.04,
              2: 0.02,
              1: 0.01,
              8: 0.0,
              7: 0.01,
              3: 0.0,
      

In [10]:
import torch

# Assuming answer_percentages is structured like: {ans_id: {question_type_id: percentage, ...}, ...}
# Extract unique question type IDs and answer IDs
unique_question_types = set()
for ans_id, q_types in answer_percentages.items():
    unique_question_types.update(q_types.keys())

# Convert to list if necessary and sort (optional but helps in consistent indexing)
question_type_ids = sorted(list(unique_question_types))
answer_ids = sorted(list(answer_percentages.keys()))

# Determine the size of the matrix
m = len(question_type_ids)  # Number of question types
n = len(answer_ids)  # Number of answers

# Initialize the matrix with zeros using torch
matrix = torch.zeros((m, n), dtype=torch.float32)

In [11]:
# Create mappings for question type IDs and answer IDs to their indices
question_type_to_index = {q_id: idx for idx, q_id in enumerate(question_type_ids)}
answer_to_index = {a_id: idx for idx, a_id in enumerate(answer_ids)}

# Populate the matrix
for ans_id, q_type_percents in answer_percentages.items():
    ans_index = answer_to_index[ans_id]
    for q_type_id, percent in q_type_percents.items():
        if q_type_id in question_type_to_index:  # Ensure the question type is in our matrix
            q_type_index = question_type_to_index[q_type_id]
            matrix[q_type_index, ans_index] = percent